In [1]:
import os
os.chdir('../')
%pwd

'D:\\projects\\Project-WineQualityDataset'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    x_train_data_path: Path
    y_train_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float

In [3]:
from WineQuality.constants import * 
from WineQuality.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self , config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH , schema_filepath = SCHEMA_FILE_PATH,):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            x_train_data_path = config.x_train_data_path,
            y_train_data_path = config.y_train_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
        )
        return model_trainer_config
        

In [10]:
import os
from WineQuality import logger
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib


class ModelTrainer:
    def __init__(self, config = ModelTrainerConfig):
        self.config = config

    def train(self):
        X_train = pd.read_csv(self.config.x_train_data_path)
        Y_train = pd.read_csv(self.config.y_train_data_path)

        model = ElasticNet(alpha = self.config.alpha, l1_ratio = self.config.l1_ratio, random_state=42)
        model.fit(X_train, Y_train)

        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        logger.info(f"Model: {self.config.model_name} trained and saved in {self.config.root_dir}")

In [11]:
try:
    config = ConfigurationManager()
    model_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_config)
    model_trainer.train()
except Exception as e:
    raise e
    

[2023-09-30 12:21:51,693: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-30 12:21:51,695: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-30 12:21:51,699: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-30 12:21:51,700: INFO: common: Directory Created: artifacts]
[2023-09-30 12:21:51,702: INFO: common: Directory Created: artifacts/model_trainer]
[2023-09-30 12:21:51,719: INFO: 384018624: Model: model.joblib trained and saved in artifacts/model_trainer]
